In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import seaborn as sns
from tensorflow import keras

from tqdm import tqdm 
from glob import glob
from skimage.io import imread
from skimage.measure import label, regionprops_table
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.optimizers import Adam
from skimage import io
from keras import backend as K
from tensorflow_addons.metrics import RSquare
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError
from skimage import data, color
from skimage.transform import resize
from commons import plot

2023-02-11 02:45:19.561361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 02:45:19.754871: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-11 02:45:19.754885: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-11 02:45:20.342832: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def print_evaluate(true, predicted):
    
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    
def run_all_regressions(X_train, X_test, Y_train, Y_test, regs):

    for name, model in regs.items():

        model.fit(X_train, Y_train)
        
        print(f'\n-----{name}------')
        print('[Train] -------------')
        print_evaluate(Y_train, model.predict(X_train))

        print("\n")
        
        print('[Test] --------------')
        print_evaluate(Y_test, model.predict(X_test))

def create_dataset(path_images):
    anilhas = []
    paths = []
    anilhas_merged = []
    paths_merged = []

    for index in glob(f"{path_images}/*"):
        anilha, *_ = re.findall(r'\d+', index)
        all_broilers = glob(index + "/**")

        anilhas.append([anilha] * len(all_broilers))
        paths.append(all_broilers)

    for anilha, path in zip(anilhas, paths):
        anilhas_merged = np.concatenate([anilhas_merged, anilha])
        paths_merged = np.concatenate([paths_merged, path])
        
    return anilhas_merged.astype(int), paths_merged

In [4]:
def labeling_datatset(anilhas, paths):
    
    df_weights = pd.read_csv("annotations.csv")

    df_out = pd.DataFrame({
        "label": anilhas,
        "path": paths,
        "weight": -1
    }) 

    for (index, weight) in zip(df_weights.id,  df_weights.weight):
        df_out.weight.iloc[df_out[df_out.label == index].index] = weight
    
    return df_out

def loading_dataset_images(df, x_size=120, y_size=120):
    return np.array([resize(imread(path), (x_size, y_size), anti_aliasing=True) 
                     for path in df.path])

In [5]:
x_size, y_size = 512 // 2, 424 // 2

In [6]:
#importe imagens de treino
anilhas_train, paths_train = create_dataset("outputs/broilers_with_neck")
df_train = labeling_datatset(anilhas_train, paths_train)

images_train =  loading_dataset_images(df_train, x_size, y_size)

#Importe imagens de teste
paths_test = glob("selecionadas/*")
anilhas_test = list(map(lambda i: int(re.findall(r'\d+', i)[0]), paths_test))

df_test = labeling_datatset(anilhas_test, paths_test)
images_test =  loading_dataset_images(df_test, x_size, y_size)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images_train, 
                                                    anilhas_train, 
                                                    test_size=0.3,
                                                    random_state=0)

# X_train, X_test, y_train, y_test = images_train, images_test, anilhas_train, anilhas_test

In [8]:
N_TRAIN = len(images_train)
BATCH_SIZE = 8
EPOCHS=30
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [9]:
from keras.regularizers import l2

model = keras.models.Sequential([
    keras.layers.Conv2D(6,
                        (5, 5),
                        activation='relu',
                        padding="same",
                        kernel_regularizer=l2(0.01), 
                        bias_regularizer=l2(0.01), 
                        input_shape=(x_size, y_size, 1)),
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(16,
                        (5, 5),
                        activation='relu', 
                        padding="same",
                        kernel_regularizer=l2(0.01),
                        bias_regularizer=l2(0.01)),
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    #FC
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(84, activation='relu'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=1, activation="linear")
])

2023-02-11 02:45:26.116378: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-11 02:45:26.116397: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-11 02:45:26.116411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mpgxc): /proc/driver/nvidia/version does not exist
2023-02-11 02:45:26.116781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 106, 6)       156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 53, 6)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 53, 6)        24        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 53, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 26, 16)       0         
 2D)                                                             
                                                        

In [11]:
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH * 1000,
    decay_rate=1,
    staircase=False)

model.compile(loss="mean_squared_error", 
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=[RSquare()])

In [12]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)

history = model.fit(X_train,
                    y_train,
                    validation_split=0.1,
                    shuffle=True,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE, 
                    callbacks=[early_stop])

Epoch 1/30
71/71 [==============================] - 4s 46ms/step - loss: 8551.3896 - r_square: -2.3259 - val_loss: 9424.2949 - val_r_square: -3.4147
Epoch 2/30
71/71 [==============================] - 3s 41ms/step - loss: 8112.4883 - r_square: -2.1552 - val_loss: 9436.7266 - val_r_square: -3.4205
Epoch 3/30
71/71 [==============================] - 3s 41ms/step - loss: 7495.5610 - r_square: -1.9152 - val_loss: 9361.0439 - val_r_square: -3.3851
Epoch 4/30
71/71 [==============================] - 3s 41ms/step - loss: 6557.6816 - r_square: -1.5505 - val_loss: 11322.2041 - val_r_square: -4.3038
Epoch 5/30
71/71 [==============================] - 3s 41ms/step - loss: 5394.4634 - r_square: -1.0980 - val_loss: 10391.9922 - val_r_square: -3.8680
Epoch 6/30
71/71 [==============================] - 3s 40ms/step - loss: 4428.7856 - r_square: -0.7225 - val_loss: 7991.4790 - val_r_square: -2.7435
Epoch 7/30
71/71 [==============================] - 3s 41ms/step - loss: 3571.9417 - r_square: -0.3892 -

In [13]:
y_train_preds = model.predict(np.asarray(X_train)).reshape(-1)
print_evaluate(y_train, y_train_preds)

20/20 [==============================] - 0s 12ms/step
MAE: 9.859236488327051
MSE: 244.5280890523883
RMSE: 15.637393934169092
R2 Square 0.9033833856350846


In [14]:
y_test_preds = model.predict(np.asarray(X_test)).reshape(-1)
print_evaluate(y_test, y_test_preds)

9/9 [==============================] - 0s 12ms/step
MAE: 36.0234725736774
MSE: 2186.2943706345213
RMSE: 46.757826838236625
R2 Square 0.15213092863152344
